<a href="https://colab.research.google.com/github/minh041104/SeminarFinal/blob/main/seminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
!pip install nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TGDD\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TGDD\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\TGDD\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Thu thập dữ liệu

Thu thập dữ liệu từ Kaggle. Đường dẫn: https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

Giới thiệu về tập dữ liệu:
1. Nguồn gốc:
AG là tập hợp của hơn triệu bài báo. Các bài viết tin tức được tổng hợp từ hơn 2000 nguồn tin tức trong hơn một năm.
2. Mô tả:
Tập dữ liệu phân loại chủ đề tin tức của AG được xây dựng bằng cách chọn 4 lớp lớn nhất trong bộ dữ liệu gốc. Mỗi lớp chứa 30,000 mẫu huấn luyện và 1,900. Tổng số dữ liệu trong tập train là 120,000 và trong tập test là 7,600.

Bộ dữ liệu gồm có 3 cột:
- Class Index: Gồm có 4 loại. 1-World, 2-Sport, 3-Business, 4-Sci/Tech.
- Title: Chứa tiêu đề của các bài báo.
- Description: Chứa mô tả của các bài báo.

# Xử lý dữ liệu

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
train_data

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


## Kiểm tra dữ liệu thiếu

In [ ]:
train_data.isna().sum()

Class Index    0
Title          0
Description    0
dtype: int64

In [ ]:
test_data.isna().sum()

Class Index    0
Title          0
Description    0
dtype: int64

In [ ]:
train_data.isnull().sum()

Class Index    0
Title          0
Description    0
dtype: int64

In [ ]:
test_data.isnull().sum()

Class Index    0
Title          0
Description    0
dtype: int64

## Kết hợp hai cột Title và Description thành một cột

Ghép hai cột này lại để có thể có được ngữ cảnh đầy đủ hơn. Title cung cấp nội dung ngắn gọn, Description cung cấp thêm vài thông tin chi tiết. Khi kết hợp lại giúp mô hình có thêm nhiều thông tin hơn để dự đoán

In [ ]:
train_data['CombinedText'] = train_data['Title'] + ' - ' + train_data['Description']
test_data['CombinedText'] = test_data['Title'] + ' - ' + test_data['Description']

In [ ]:
train_data

,Class Index,Title,Description,CombinedText
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","Oil prices soar to all-time record, posing new..."
...,...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...,Pakistan's Musharraf Says Won't Quit as Army C...
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...,Renteria signing a top-shelf deal - Red Sox ge...
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...,Saban not going to Dolphins yet - The Miami Do...
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...,Today's NFL games - PITTSBURGH at NY GIANTS Ti...




```
# Định dạng của đoạn này là mã
```

## Chuyển về chữ thường

In [ ]:
train_data['CombinedText'] = train_data['CombinedText'].str.lower()
test_data['CombinedText'] = test_data['CombinedText'].str.lower()

## Loại bỏ URL

In [ ]:
def url_remove(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub('', text)

train_data['CombinedText'] = train_data['CombinedText'].apply(url_remove)
test_data['CombinedText'] = test_data['CombinedText'].apply(url_remove)

## Loại bỏ hashtag

In [ ]:
def remove_hashtag(text):
    hashtag_pattern = re.compile(r'#\w+')
    cleaned_text = hashtag_pattern.sub('', text)
    return cleaned_text

train_data['CombinedText'] = train_data['CombinedText'].apply(remove_hashtag)
test_data['CombinedText'] = test_data['CombinedText'].apply(remove_hashtag)

## Loại bỏ thẻ html

In [ ]:
def remove_html_tag(text):
    html = re.compile('<.*?')
    cleaned_text = html.sub('', text)
    return cleaned_text

train_data['CombinedText'] = train_data['CombinedText'].apply(remove_html_tag)
test_data['CombinedText'] = test_data['CombinedText'].apply(remove_html_tag)

## Mở rộng các từ viết tắt

In [ ]:
contractions = {"ain't": "are not","'s":" is","aren't": "are not", "n't" : "not"}

def expand_contractions(text, contractions):
    for contraction, expansion in contractions.items():
        text = text.replace(contraction, expansion)
    return text

train_data['CombinedText'] = train_data['CombinedText'].apply(lambda x: expand_contractions(x, contractions))
test_data['CombinedText'] = test_data['CombinedText'].apply(lambda x: expand_contractions(x, contractions))

## Loại bỏ các dấu câu

In [ ]:
def remove_punctuation(text):
    punctuation_pattern = re.compile(r'[^\w\s]')
    cleaned_text = punctuation_pattern.sub('', text)
    return cleaned_text

train_data['CombinedText'] = train_data['CombinedText'].apply(remove_punctuation)
test_data['CombinedText'] = test_data['CombinedText'].apply(remove_punctuation)

## Loại bỏ các chữ số

In [ ]:
def remove_digits(text):
    digits_pattern = re.compile(r'\d+')
    cleaned_text = digits_pattern.sub('', text)
    return cleaned_text

train_data['CombinedText'] = train_data['CombinedText'].apply(remove_digits)
test_data['CombinedText'] = test_data['CombinedText'].apply(remove_digits)

## Loại bỏ stopwords

In [ ]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

train_data['CombinedText'] = train_data['CombinedText'].apply(remove_stopwords)
test_data['CombinedText'] = test_data['CombinedText'].apply(remove_stopwords)

## Tokenization

In [ ]:
train_data['CombinedText'] = train_data['CombinedText'].apply(word_tokenize)
test_data['CombinedText'] = test_data['CombinedText'].apply(word_tokenize)

## Stemming và Lemmatization

- Stemming
    - Là kỹ thuật dùng để biến đổi 1 từ về dạng gốc bằng cách cực kỳ đơn giản là loại bỏ 1 số ký tự nằm ở cuối từ mà nó nghĩ rằng là biến thể của từ.
    - Ví dụ như chúng ta thấy các từ như walked, walking, walks chỉ khác nhau là ở những ký tự cuối cùng, bằng cách bỏ đi các hậu tố -ed, -ing hoặc -s, chúng ta sẽ được từ nguyên gốc là walk.
    - Nó có một số nhược điểm như khi gặp từ goes sẽ được stem thành goe trong khi stem của go là go. Kết quả của hai từ goes và go sau khi stem không giống nhau. Nhược điểm khác như khi gặp từ bất quy tắc thì không thể đưa went về dạng go.
- Lemmatization
    - Lemmatization cũng như Stemming nhưng nó thực hiện việc chuẩn hóa từ thành từ gốc một cách chính xác hơn bằng cách sử dụng kiến thức ngôn ngữ và từ điển.
    - Quá trình này thường yêu cầu sử dụng thông tin ngữ cảnh và từ điển để xác định từ gốc của một từ.
    - Điều này sẽ đảm bảo rằng các từ như “goes“, “went” và “go” sẽ chắc chắn có kết quả trả về là như nhau.
    

In [ ]:
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def stem_words(words):
    return [ps.stem(word) for word in words]

def lemmatize_words(words):
    return [lemmatizer.lemmatize(word) for word in words]

train_data['CombinedText'] = train_data['CombinedText'].apply(stem_words)
test_data['CombinedText'] = test_data['CombinedText'].apply(stem_words)

train_data['CombinedText'] = train_data['CombinedText'].apply(lemmatize_words)
test_data['CombinedText'] = test_data['CombinedText'].apply(lemmatize_words)

# Trích xuất dữ liệu

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

train_data['Text'] = train_data['CombinedText'].apply(lambda x: ' '.join(x))
test_data['Text'] = test_data['CombinedText'].apply(lambda x: ' '.join(x))

vectorizer = TfidfVectorizer()
tfidf_matrix_train = vectorizer.fit_transform(train_data['Text'])
tfidf_matrix_test = vectorizer.transform(test_data['Text'])

## Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

bow_matrix_train = vectorizer.fit_transform(train_data['Text'])
bow_matrix_test = vectorizer.transform(test_data['Text'])

# Model

In [ ]:
from scipy.sparse import hstack

# Kết hợp ma trận TF-IDF và BoW cho tập huấn luyện
combined_matrix_train = hstack([tfidf_matrix_train, bow_matrix_train])

# Kết hợp ma trận TF-IDF và BoW cho tập kiểm tra
combined_matrix_test = hstack([tfidf_matrix_test, bow_matrix_test])

## KNN

### 5000 training data

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 5000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 5000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_5000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình KNN với số láng giềng k=5 (có thể điều chỉnh)
knn_model = KNeighborsClassifier(n_neighbors=5)

# Huấn luyện mô hình trên mẫu dữ liệu mới chỉ chứa 5000 mẫu
knn_model.fit(combined_matrix_train_5000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_knn = knn_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_knn = accuracy_score(test_data['Class Index'], y_pred_knn)
print("Accuracy (KNN):", accuracy_knn)

Accuracy (KNN): 0.5668421052631579


### 10000 training data

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 10000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 10000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_10000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình KNN với số láng giềng k=5 (có thể điều chỉnh)
knn_model = KNeighborsClassifier(n_neighbors=5)

# Huấn luyện mô hình trên mẫu dữ liệu mới chỉ chứa 10000 mẫu
knn_model.fit(combined_matrix_train_10000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_knn = knn_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_knn = accuracy_score(test_data['Class Index'], y_pred_knn)
print("Accuracy (KNN):", accuracy_knn)


Accuracy (KNN): 0.5747368421052632


### 20000 training data

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 20000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 20000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_20000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình KNN với số láng giềng k=5 (có thể điều chỉnh)
knn_model = KNeighborsClassifier(n_neighbors=5)

# Huấn luyện mô hình trên mẫu dữ liệu mới chỉ chứa 20000 mẫu
knn_model.fit(combined_matrix_train_20000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_knn = knn_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_knn = accuracy_score(test_data['Class Index'], y_pred_knn)
print("Accuracy (KNN):", accuracy_knn)


Accuracy (KNN): 0.7097368421052631


### 50000 training data

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 50000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 50000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_50000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình KNN với số láng giềng k=5 (có thể điều chỉnh)
knn_model = KNeighborsClassifier(n_neighbors=5)

# Huấn luyện mô hình trên mẫu dữ liệu mới chỉ chứa 50000 mẫu
knn_model.fit(combined_matrix_train_50000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_knn = knn_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_knn = accuracy_score(test_data['Class Index'], y_pred_knn)
print("Accuracy (KNN):", accuracy_knn)


Accuracy (KNN): 0.7457894736842106


### 100000 training data

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 100000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 100000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_100000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình KNN với số láng giềng k=5 (có thể điều chỉnh)
knn_model = KNeighborsClassifier(n_neighbors=5)

# Huấn luyện mô hình trên mẫu dữ liệu mới chỉ chứa 100000 mẫu
knn_model.fit(combined_matrix_train_100000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_knn = knn_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_knn = accuracy_score(test_data['Class Index'], y_pred_knn)
print("Accuracy (KNN):", accuracy_knn)


Accuracy (KNN): 0.6634210526315789


### 120000 training data

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 120000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 120000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_120000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình KNN với số láng giềng k=5 (có thể điều chỉnh)
knn_model = KNeighborsClassifier(n_neighbors=5)

# Huấn luyện mô hình trên mẫu dữ liệu mới chỉ chứa 120000 mẫu
knn_model.fit(combined_matrix_train_120000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_knn = knn_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_knn = accuracy_score(test_data['Class Index'], y_pred_knn)
print("Accuracy (KNN):", accuracy_knn)


Accuracy (KNN): 0.6876315789473684


## Logistic Regression

### 5000 training data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 5000 chỉ mục từ cả ma trận kết hợp huấn luyện và kiểm tra
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 5000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp
combined_matrix_train_5000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Logistic Regression
logistic_model = LogisticRegression(max_iter=500)

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 5000 mẫu
logistic_model.fit(combined_matrix_train_5000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred = logistic_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy = accuracy_score(test_data['Class Index'], y_pred)
print("Accuracy (Combined TF-IDF + BoW):", accuracy)

Accuracy (Combined TF-IDF + BoW): 0.8611842105263158


### 10000 training data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 10000 chỉ mục từ cả ma trận kết hợp huấn luyện và kiểm tra
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 10000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp
combined_matrix_train_10000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Logistic Regression
logistic_model = LogisticRegression(max_iter=500)

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 10000 mẫu
logistic_model.fit(combined_matrix_train_10000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred = logistic_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy = accuracy_score(test_data['Class Index'], y_pred)
print("Accuracy (Combined TF-IDF + BoW):", accuracy)

Accuracy (Combined TF-IDF + BoW): 0.87


### 20000 training data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 20000 chỉ mục từ cả ma trận kết hợp huấn luyện và kiểm tra
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 20000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp
combined_matrix_train_20000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Logistic Regression
logistic_model = LogisticRegression(max_iter=500)

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 20000 mẫu
logistic_model.fit(combined_matrix_train_20000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred = logistic_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy = accuracy_score(test_data['Class Index'], y_pred)
print("Accuracy (Combined TF-IDF + BoW):", accuracy)

Accuracy (Combined TF-IDF + BoW): 0.8882894736842105


### 50000 training data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 50000 chỉ mục từ cả ma trận kết hợp huấn luyện và kiểm tra
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 50000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp
combined_matrix_train_50000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Logistic Regression
logistic_model = LogisticRegression(max_iter=500)

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 50000 mẫu
logistic_model.fit(combined_matrix_train_50000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred = logistic_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy = accuracy_score(test_data['Class Index'], y_pred)
print("Accuracy (Combined TF-IDF + BoW):", accuracy)

Accuracy (Combined TF-IDF + BoW): 0.8961842105263158


### 100000 training data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 100000 chỉ mục từ cả ma trận kết hợp huấn luyện và kiểm tra
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 100000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp
combined_matrix_train_100000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Logistic Regression
logistic_model = LogisticRegression(max_iter=500)

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 100000 mẫu
logistic_model.fit(combined_matrix_train_100000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred = logistic_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy = accuracy_score(test_data['Class Index'], y_pred)
print("Accuracy (Combined TF-IDF + BoW):", accuracy)

Accuracy (Combined TF-IDF + BoW): 0.9014473684210527


### 120000 training data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 120000 chỉ mục từ cả ma trận kết hợp huấn luyện và kiểm tra
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 120000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp
combined_matrix_train_120000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Logistic Regression
logistic_model = LogisticRegression(max_iter=500)

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 120000 mẫu
logistic_model.fit(combined_matrix_train_120000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred = logistic_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy = accuracy_score(test_data['Class Index'], y_pred)
print("Accuracy (Combined TF-IDF + BoW):", accuracy)

Accuracy (Combined TF-IDF + BoW): 0.9064473684210527


## Naive bayes

### 5000 training data

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 5000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 5000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_5000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Naive Bayes
nb_model = MultinomialNB()

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 5000 mẫu
nb_model.fit(combined_matrix_train_5000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred = nb_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy = accuracy_score(test_data['Class Index'], y_pred)
print("Accuracy (Combined TF-IDF + BoW):", accuracy)


Accuracy (Combined TF-IDF + BoW): 0.8786842105263157


### 10000 training data

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 10000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 10000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_10000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Naive Bayes
nb_model = MultinomialNB()

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 10000 mẫu
nb_model.fit(combined_matrix_train_10000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred = nb_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy = accuracy_score(test_data['Class Index'], y_pred)
print("Accuracy (Combined TF-IDF + BoW):", accuracy)


Accuracy (Combined TF-IDF + BoW): 0.8877631578947368


### 20000 training data

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 20000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 20000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_20000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Naive Bayes
nb_model = MultinomialNB()

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 20000 mẫu
nb_model.fit(combined_matrix_train_20000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred = nb_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy = accuracy_score(test_data['Class Index'], y_pred)
print("Accuracy (Combined TF-IDF + BoW):", accuracy)


Accuracy (Combined TF-IDF + BoW): 0.8961842105263158


### 50000 training data

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 50000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 50000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_50000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Naive Bayes
nb_model = MultinomialNB()

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 50000 mẫu
nb_model.fit(combined_matrix_train_50000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred = nb_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy = accuracy_score(test_data['Class Index'], y_pred)
print("Accuracy (Combined TF-IDF + BoW):", accuracy)


Accuracy (Combined TF-IDF + BoW): 0.8990789473684211


### 100000 training data

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 100000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 100000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_100000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Naive Bayes
nb_model = MultinomialNB()

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 100000 mẫu
nb_model.fit(combined_matrix_train_100000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred = nb_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy = accuracy_score(test_data['Class Index'], y_pred)
print("Accuracy (Combined TF-IDF + BoW):", accuracy)


Accuracy (Combined TF-IDF + BoW): 0.9013157894736842


### 120000 training data

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 120000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 120000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_120000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Naive Bayes
nb_model = MultinomialNB()

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 120000 mẫu
nb_model.fit(combined_matrix_train_120000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred = nb_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy = accuracy_score(test_data['Class Index'], y_pred)
print("Accuracy (Combined TF-IDF + BoW):", accuracy)


Accuracy (Combined TF-IDF + BoW): 0.901578947368421


## Random Forest

- Do tài nguyên hạn chế nên đã tối ưu tham số để giảm thời gian chạy:
    - n_estimators=50,          Số lượng cây nhỏ hơn
    - max_depth=10,             Giới hạn độ sâu của cây

### 5000 training data

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 5000 chỉ mục từ ma trận kết hợp huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 5000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_5000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Random Forest với các tham số tối ưu
rf_model = RandomForestClassifier(
    n_estimators=50,          # Số lượng cây nhỏ hơn
    max_depth=10,             # Giới hạn độ sâu của cây
    max_features='sqrt',      # Số lượng đặc trưng xem xét khi chia nhánh
    random_state=42,
    n_jobs=-1                 # Sử dụng tất cả các CPU cores có sẵn
)

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 5000 mẫu
rf_model.fit(combined_matrix_train_5000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_rf = rf_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_rf = accuracy_score(test_data['Class Index'], y_pred_rf)
print("Accuracy (Random Forest):", accuracy_rf)


Accuracy (Random Forest): 0.7684210526315789


### 10000 training data

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 10000 chỉ mục từ ma trận kết hợp huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 10000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_10000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Random Forest với các tham số tối ưu
rf_model = RandomForestClassifier(
    n_estimators=50,          # Số lượng cây nhỏ hơn
    max_depth=10,             # Giới hạn độ sâu của cây
    max_features='sqrt',      # Số lượng đặc trưng xem xét khi chia nhánh
    random_state=42,
    n_jobs=-1                 # Sử dụng tất cả các CPU cores có sẵn
)

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 10000 mẫu
rf_model.fit(combined_matrix_train_10000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_rf = rf_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_rf = accuracy_score(test_data['Class Index'], y_pred_rf)
print("Accuracy (Random Forest):", accuracy_rf)


Accuracy (Random Forest): 0.7885526315789474


### 20000 training data

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 20000 chỉ mục từ ma trận kết hợp huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 20000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_20000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Random Forest với các tham số tối ưu
rf_model = RandomForestClassifier(
    n_estimators=50,          # Số lượng cây nhỏ hơn
    max_depth=10,             # Giới hạn độ sâu của cây
    max_features='sqrt',      # Số lượng đặc trưng xem xét khi chia nhánh
    random_state=42,
    n_jobs=-1                 # Sử dụng tất cả các CPU cores có sẵn
)

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 20000 mẫu
rf_model.fit(combined_matrix_train_20000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_rf = rf_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_rf = accuracy_score(test_data['Class Index'], y_pred_rf)
print("Accuracy (Random Forest):", accuracy_rf)


Accuracy (Random Forest): 0.7898684210526316


### 50000 training data

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 50000 chỉ mục từ ma trận kết hợp huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 50000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_50000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Random Forest với các tham số tối ưu
rf_model = RandomForestClassifier(
    n_estimators=50,          # Số lượng cây nhỏ hơn
    max_depth=10,             # Giới hạn độ sâu của cây
    max_features='sqrt',      # Số lượng đặc trưng xem xét khi chia nhánh
    random_state=42,
    n_jobs=-1                 # Sử dụng tất cả các CPU cores có sẵn
)

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 50000 mẫu
rf_model.fit(combined_matrix_train_50000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_rf = rf_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_rf = accuracy_score(test_data['Class Index'], y_pred_rf)
print("Accuracy (Random Forest):", accuracy_rf)


Accuracy (Random Forest): 0.796578947368421


### 100000 training data

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 100000 chỉ mục từ ma trận kết hợp huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 100000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_100000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Random Forest với các tham số tối ưu
rf_model = RandomForestClassifier(
    n_estimators=50,          # Số lượng cây nhỏ hơn
    max_depth=10,             # Giới hạn độ sâu của cây
    max_features='sqrt',      # Số lượng đặc trưng xem xét khi chia nhánh
    random_state=42,
    n_jobs=-1                 # Sử dụng tất cả các CPU cores có sẵn
)

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 100000 mẫu
rf_model.fit(combined_matrix_train_100000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_rf = rf_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_rf = accuracy_score(test_data['Class Index'], y_pred_rf)
print("Accuracy (Random Forest):", accuracy_rf)


Accuracy (Random Forest): 0.7988157894736843


### 120000 training data

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 120000 chỉ mục từ ma trận kết hợp huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 120000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_120000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình Random Forest với các tham số tối ưu
rf_model = RandomForestClassifier(
    n_estimators=50,          # Số lượng cây nhỏ hơn
    max_depth=10,             # Giới hạn độ sâu của cây
    max_features='sqrt',      # Số lượng đặc trưng xem xét khi chia nhánh
    random_state=42,
    n_jobs=-1                 # Sử dụng tất cả các CPU cores có sẵn
)

# Huấn luyện mô hình trên ma trận dữ liệu mới chỉ chứa 120000 mẫu
rf_model.fit(combined_matrix_train_120000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_rf = rf_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_rf = accuracy_score(test_data['Class Index'], y_pred_rf)
print("Accuracy (Random Forest):", accuracy_rf)


Accuracy (Random Forest): 0.8110526315789474


## Quadratic Discriminant Analysis (QDA)

### 5000 training data

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 5000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 5000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_5000 = combined_matrix_train[train_random_indices]

# Khởi tạo và huấn luyện mô hình TruncatedSVD để giảm số lượng đặc trưng
n_components = 100  # Số lượng thành phần có thể điều chỉnh dựa trên dữ liệu của bạn
svd = TruncatedSVD(n_components=n_components)
combined_matrix_train_svd = svd.fit_transform(combined_matrix_train_5000)

# Áp dụng TruncatedSVD cho tập kiểm tra
combined_matrix_test_svd = svd.transform(combined_matrix_test)

# Khởi tạo mô hình Quadratic Discriminant Analysis (QDA)
qda_model = QuadraticDiscriminantAnalysis()

# Huấn luyện mô hình trên dữ liệu đã áp dụng TruncatedSVD
qda_model.fit(combined_matrix_train_svd, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_qda = qda_model.predict(combined_matrix_test_svd)

# Đánh giá mô hình trên tập kiểm tra
accuracy_qda = accuracy_score(test_data['Class Index'], y_pred_qda)
print("Accuracy (QDA):", accuracy_qda)


Accuracy (QDA): 0.761578947368421


### 10000 training data

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 10000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 10000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_10000 = combined_matrix_train[train_random_indices]

# Khởi tạo và huấn luyện mô hình TruncatedSVD để giảm số lượng đặc trưng
n_components = 100  # Số lượng thành phần có thể điều chỉnh dựa trên dữ liệu của bạn
svd = TruncatedSVD(n_components=n_components)
combined_matrix_train_svd = svd.fit_transform(combined_matrix_train_10000)

# Áp dụng TruncatedSVD cho tập kiểm tra
combined_matrix_test_svd = svd.transform(combined_matrix_test)

# Khởi tạo mô hình Quadratic Discriminant Analysis (QDA)
qda_model = QuadraticDiscriminantAnalysis()

# Huấn luyện mô hình trên dữ liệu đã áp dụng TruncatedSVD
qda_model.fit(combined_matrix_train_svd, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_qda = qda_model.predict(combined_matrix_test_svd)

# Đánh giá mô hình trên tập kiểm tra
accuracy_qda = accuracy_score(test_data['Class Index'], y_pred_qda)
print("Accuracy (QDA):", accuracy_qda)


Accuracy (QDA): 0.7676315789473684


### 20000 training data

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 20000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 20000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_20000 = combined_matrix_train[train_random_indices]

# Khởi tạo và huấn luyện mô hình TruncatedSVD để giảm số lượng đặc trưng
n_components = 100  # Số lượng thành phần có thể điều chỉnh dựa trên dữ liệu của bạn
svd = TruncatedSVD(n_components=n_components)
combined_matrix_train_svd = svd.fit_transform(combined_matrix_train_20000)

# Áp dụng TruncatedSVD cho tập kiểm tra
combined_matrix_test_svd = svd.transform(combined_matrix_test)

# Khởi tạo mô hình Quadratic Discriminant Analysis (QDA)
qda_model = QuadraticDiscriminantAnalysis()

# Huấn luyện mô hình trên dữ liệu đã áp dụng TruncatedSVD
qda_model.fit(combined_matrix_train_svd, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_qda = qda_model.predict(combined_matrix_test_svd)

# Đánh giá mô hình trên tập kiểm tra
accuracy_qda = accuracy_score(test_data['Class Index'], y_pred_qda)
print("Accuracy (QDA):", accuracy_qda)


Accuracy (QDA): 0.7734210526315789


### 50000 training data

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 50000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 50000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_50000 = combined_matrix_train[train_random_indices]

# Khởi tạo và huấn luyện mô hình TruncatedSVD để giảm số lượng đặc trưng
n_components = 100  # Số lượng thành phần có thể điều chỉnh dựa trên dữ liệu của bạn
svd = TruncatedSVD(n_components=n_components)
combined_matrix_train_svd = svd.fit_transform(combined_matrix_train_50000)

# Áp dụng TruncatedSVD cho tập kiểm tra
combined_matrix_test_svd = svd.transform(combined_matrix_test)

# Khởi tạo mô hình Quadratic Discriminant Analysis (QDA)
qda_model = QuadraticDiscriminantAnalysis()

# Huấn luyện mô hình trên dữ liệu đã áp dụng TruncatedSVD
qda_model.fit(combined_matrix_train_svd, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_qda = qda_model.predict(combined_matrix_test_svd)

# Đánh giá mô hình trên tập kiểm tra
accuracy_qda = accuracy_score(test_data['Class Index'], y_pred_qda)
print("Accuracy (QDA):", accuracy_qda)


Accuracy (QDA): 0.765921052631579


### 100000 training data

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 100000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 100000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_100000 = combined_matrix_train[train_random_indices]

# Khởi tạo và huấn luyện mô hình TruncatedSVD để giảm số lượng đặc trưng
n_components = 100  # Số lượng thành phần có thể điều chỉnh dựa trên dữ liệu của bạn
svd = TruncatedSVD(n_components=n_components)
combined_matrix_train_svd = svd.fit_transform(combined_matrix_train_100000)

# Áp dụng TruncatedSVD cho tập kiểm tra
combined_matrix_test_svd = svd.transform(combined_matrix_test)

# Khởi tạo mô hình Quadratic Discriminant Analysis (QDA)
qda_model = QuadraticDiscriminantAnalysis()

# Huấn luyện mô hình trên dữ liệu đã áp dụng TruncatedSVD
qda_model.fit(combined_matrix_train_svd, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_qda = qda_model.predict(combined_matrix_test_svd)

# Đánh giá mô hình trên tập kiểm tra
accuracy_qda = accuracy_score(test_data['Class Index'], y_pred_qda)
print("Accuracy (QDA):", accuracy_qda)


Accuracy (QDA): 0.770921052631579


### 120000 training data

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score

# Chọn ngẫu nhiên 120000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 120000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_120000 = combined_matrix_train[train_random_indices]

# Khởi tạo và huấn luyện mô hình TruncatedSVD để giảm số lượng đặc trưng
n_components = 100  # Số lượng thành phần có thể điều chỉnh dựa trên dữ liệu của bạn
svd = TruncatedSVD(n_components=n_components)
combined_matrix_train_svd = svd.fit_transform(combined_matrix_train_120000)

# Áp dụng TruncatedSVD cho tập kiểm tra
combined_matrix_test_svd = svd.transform(combined_matrix_test)

# Khởi tạo mô hình Quadratic Discriminant Analysis (QDA)
qda_model = QuadraticDiscriminantAnalysis()

# Huấn luyện mô hình trên dữ liệu đã áp dụng TruncatedSVD
qda_model.fit(combined_matrix_train_svd, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_qda = qda_model.predict(combined_matrix_test_svd)

# Đánh giá mô hình trên tập kiểm tra
accuracy_qda = accuracy_score(test_data['Class Index'], y_pred_qda)
print("Accuracy (QDA):", accuracy_qda)


Accuracy (QDA): 0.7714473684210527


## SVM

Do tài nguyên có hạn nên dùng Stochastic Gradient Descent (SGD) thay vì SVM truyền thống có thể là cần thiết để đảm bảo rằng mô hình có thể hoạt động hiệu quả trên tài nguyên có hạn.

### 5000 training data

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 5000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 5000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_5000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình SGDClassifier với SVM tuyến tính
sgd_model = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3)

# Huấn luyện mô hình trên mẫu dữ liệu mới chỉ chứa 5000 mẫu
sgd_model.fit(combined_matrix_train_5000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_sgd = sgd_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_sgd = accuracy_score(test_data['Class Index'], y_pred_sgd)
print("Accuracy (SVM with SGD):", accuracy_sgd)


Accuracy (SVM with SGD): 0.8465789473684211


### 10000 training data

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 10000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 10000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_10000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình SGDClassifier với SVM tuyến tính
sgd_model = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3)

# Huấn luyện mô hình trên mẫu dữ liệu mới chỉ chứa 10000 mẫu
sgd_model.fit(combined_matrix_train_10000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_sgd = sgd_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_sgd = accuracy_score(test_data['Class Index'], y_pred_sgd)
print("Accuracy (SVM with SGD):", accuracy_sgd)


Accuracy (SVM with SGD): 0.8619736842105263


### 20000 training data

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 20000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 20000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_20000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình SGDClassifier với SVM tuyến tính
sgd_model = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3)

# Huấn luyện mô hình trên mẫu dữ liệu mới chỉ chứa 20000 mẫu
sgd_model.fit(combined_matrix_train_20000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_sgd = sgd_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_sgd = accuracy_score(test_data['Class Index'], y_pred_sgd)
print("Accuracy (SVM with SGD):", accuracy_sgd)


Accuracy (SVM with SGD): 0.8775


### 50000 training data

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 50000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 50000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_50000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình SGDClassifier với SVM tuyến tính
sgd_model = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3)

# Huấn luyện mô hình trên mẫu dữ liệu mới chỉ chứa 50000 mẫu
sgd_model.fit(combined_matrix_train_50000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_sgd = sgd_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_sgd = accuracy_score(test_data['Class Index'], y_pred_sgd)
print("Accuracy (SVM with SGD):", accuracy_sgd)


Accuracy (SVM with SGD): 0.9085526315789474


### 100000 training data

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 100000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 100000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_100000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình SGDClassifier với SVM tuyến tính
sgd_model = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3)

# Huấn luyện mô hình trên mẫu dữ liệu mới chỉ chứa 100000 mẫu
sgd_model.fit(combined_matrix_train_100000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_sgd = sgd_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_sgd = accuracy_score(test_data['Class Index'], y_pred_sgd)
print("Accuracy (SVM with SGD):", accuracy_sgd)


Accuracy (SVM with SGD): 0.9163157894736842


### 120000 training data

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 120000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 120000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_120000 = combined_matrix_train[train_random_indices]

# Khởi tạo mô hình SGDClassifier với SVM tuyến tính
sgd_model = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3)

# Huấn luyện mô hình trên mẫu dữ liệu mới chỉ chứa 120000 mẫu
sgd_model.fit(combined_matrix_train_120000, train_data['Class Index'].iloc[train_random_indices])

# Dự đoán nhãn cho tập kiểm tra
y_pred_sgd = sgd_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_sgd = accuracy_score(test_data['Class Index'], y_pred_sgd)
print("Accuracy (SVM with SGD):", accuracy_sgd)


Accuracy (SVM with SGD): 0.9194736842105263


## Bagging

Do tài nguyên có hạn nên chỉ giới hạn số lượng mô hình base lại

### 5000 training data

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 5000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 5000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_5000 = combined_matrix_train[train_random_indices]
train_labels_5000 = train_data['Class Index'].iloc[train_random_indices]

# Khởi tạo mô hình Decision Tree base
base_model = DecisionTreeClassifier()

n_estimators = 5

# Khởi tạo mô hình Bagging với Decision Tree base và số lượng cây giảm
bagging_model = BaggingClassifier(base_model, n_estimators=n_estimators, random_state=42)

# Huấn luyện mô hình Bagging
bagging_model.fit(combined_matrix_train_5000, train_labels_5000)

# Dự đoán nhãn cho tập kiểm tra
y_pred_bagging = bagging_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_bagging = accuracy_score(test_data['Class Index'], y_pred_bagging)
print("Accuracy (Bagging):", accuracy_bagging)


Accuracy (Bagging): 0.743421052631579


### 10000 training data

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 10000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 10000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_10000 = combined_matrix_train[train_random_indices]
train_labels_10000 = train_data['Class Index'].iloc[train_random_indices]

# Khởi tạo mô hình Decision Tree base
base_model = DecisionTreeClassifier()

n_estimators = 5

# Khởi tạo mô hình Bagging với Decision Tree base và số lượng cây giảm
bagging_model = BaggingClassifier(base_model, n_estimators=n_estimators, random_state=42)

# Huấn luyện mô hình Bagging
bagging_model.fit(combined_matrix_train_10000, train_labels_10000)

# Dự đoán nhãn cho tập kiểm tra
y_pred_bagging = bagging_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_bagging = accuracy_score(test_data['Class Index'], y_pred_bagging)
print("Accuracy (Bagging):", accuracy_bagging)


Accuracy (Bagging): 0.7692105263157895


### 20000 training data

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 20000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 20000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_20000 = combined_matrix_train[train_random_indices]
train_labels_20000 = train_data['Class Index'].iloc[train_random_indices]

# Khởi tạo mô hình Decision Tree base
base_model = DecisionTreeClassifier()

n_estimators = 5

# Khởi tạo mô hình Bagging với Decision Tree base và số lượng cây giảm
bagging_model = BaggingClassifier(base_model, n_estimators=n_estimators, random_state=42)

# Huấn luyện mô hình Bagging
bagging_model.fit(combined_matrix_train_20000, train_labels_20000)

# Dự đoán nhãn cho tập kiểm tra
y_pred_bagging = bagging_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_bagging = accuracy_score(test_data['Class Index'], y_pred_bagging)
print("Accuracy (Bagging):", accuracy_bagging)


Accuracy (Bagging): 0.7886842105263158


### 50000 training data

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 50000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 50000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_50000 = combined_matrix_train[train_random_indices]
train_labels_50000 = train_data['Class Index'].iloc[train_random_indices]

# Khởi tạo mô hình Decision Tree base
base_model = DecisionTreeClassifier()

n_estimators = 5

# Khởi tạo mô hình Bagging với Decision Tree base và số lượng cây giảm
bagging_model = BaggingClassifier(base_model, n_estimators=n_estimators, random_state=42)

# Huấn luyện mô hình Bagging
bagging_model.fit(combined_matrix_train_50000, train_labels_50000)

# Dự đoán nhãn cho tập kiểm tra
y_pred_bagging = bagging_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_bagging = accuracy_score(test_data['Class Index'], y_pred_bagging)
print("Accuracy (Bagging):", accuracy_bagging)


Accuracy (Bagging): 0.8281578947368421


### 100000 training data

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 100000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 100000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_100000 = combined_matrix_train[train_random_indices]
train_labels_100000 = train_data['Class Index'].iloc[train_random_indices]

# Khởi tạo mô hình Decision Tree base
base_model = DecisionTreeClassifier()

n_estimators = 5

# Khởi tạo mô hình Bagging với Decision Tree base và số lượng cây giảm
bagging_model = BaggingClassifier(base_model, n_estimators=n_estimators, random_state=42)

# Huấn luyện mô hình Bagging
bagging_model.fit(combined_matrix_train_100000, train_labels_100000)

# Dự đoán nhãn cho tập kiểm tra
y_pred_bagging = bagging_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_bagging = accuracy_score(test_data['Class Index'], y_pred_bagging)
print("Accuracy (Bagging):", accuracy_bagging)


Accuracy (Bagging): 0.8351315789473684


### 120000 training data

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Chọn ngẫu nhiên 120000 chỉ mục từ tập dữ liệu huấn luyện
train_random_indices = np.random.choice(combined_matrix_train.shape[0], 120000, replace=False)

# Lấy các dòng tương ứng từ ma trận kết hợp huấn luyện
combined_matrix_train_120000 = combined_matrix_train[train_random_indices]
train_labels_120000 = train_data['Class Index'].iloc[train_random_indices]

# Khởi tạo mô hình Decision Tree base
base_model = DecisionTreeClassifier()

n_estimators = 5

# Khởi tạo mô hình Bagging với Decision Tree base và số lượng cây giảm
bagging_model = BaggingClassifier(base_model, n_estimators=n_estimators, random_state=42)

# Huấn luyện mô hình Bagging
bagging_model.fit(combined_matrix_train_120000, train_labels_120000)

# Dự đoán nhãn cho tập kiểm tra
y_pred_bagging = bagging_model.predict(combined_matrix_test)

# Đánh giá mô hình trên tập kiểm tra
accuracy_bagging = accuracy_score(test_data['Class Index'], y_pred_bagging)
print("Accuracy (Bagging):", accuracy_bagging)


Accuracy (Bagging): 0.8439473684210527
